In [22]:
# NEW GA functions that use only dictionaries and simplifed GA use
import numpy as np
import pandas as pd

# Defining the GA parameters
GA_params = { 'population_size': 3,
             'num_parents': 1,
             'num_generations': 1,
             'crossover_rate': 0.8,
             'mutation_rate': 0.9,
             'mutation_range': 0.1}
print(GA_params)
sequence_params_GA = dict(
    bounds_pit = {'no_pulses': (1, 1001),
                 'f_0': 250E6,
                'amplitude': (0.001, 0.21),
                'f_sweep': 12E6/4,
                'duration': (10E-6, 100E-6),
                'delay': (10E-6, 300E-6),
                'phase': 0},

    bounds_burn_back = {'no_pulses': (1, 1001),
                'f_0': (10E5, 500E6),
                'amplitude': (0.001, 0.21),
                'freq_sweep': (0.1E-6, 5E6),
                'duration': (10E-6, 100E-6),
                'delay': (10E-6, 300E-6),
                'phase': 0},

    bounds_clean = {'no_pulses': (1, 1001),
                'f_0': (10E5, 500E6),
                'amplitude': (0.001, 0.21),
                'freq_sweep': (0.1E-6, 5E6),
                'duration': (10E-6, 100E-6),
                'delay': (10E-6, 300E-6),
                'phase': 0}
)


{'population_size': 3, 'num_parents': 1, 'num_generations': 1, 'crossover_rate': 0.8, 'mutation_rate': 0.9, 'mutation_range': 0.1}


In [29]:
# Initial Population function

def initial_population(GA_params, sequence_params_GA):
    population_all = {}
    
    population_size = GA_params['population_size']
    
    for bound_key, bound_value in sequence_params_GA.items():
        population = []
        params = list(bound_value.keys())
        
        for i in range(population_size):
            individual = {}
            
            for param in params:
                bound_param = bound_value[param]
                if isinstance(bound_param, tuple) and len(bound_param) == 2:
                    lower, upper = bound_param
                    if isinstance(lower, int) and isinstance(upper, int):
                        value = np.random.randint(lower, upper)
                    else:
                        value = np.random.uniform(lower, upper)
                else:
                    value = bound_param
                
                individual[param] = value
            
            population.append(individual)
        
        population_all[bound_key] = population
    
    return population_all

# Example
sequence_params = initial_population(GA_params, sequence_params_GA)

print("\nPopulation :\n", sequence_params)


Population :
 {'bounds_pit': [{'no_pulses': 144, 'f_0': 250000000.0, 'amplitude': 0.07008784451277557, 'f_sweep': 3000000.0, 'duration': 8.871477062145104e-05, 'delay': 0.0001785806389563743, 'phase': 0}, {'no_pulses': 226, 'f_0': 250000000.0, 'amplitude': 0.08993842451064414, 'f_sweep': 3000000.0, 'duration': 3.679000837093852e-05, 'delay': 0.0001443405480765229, 'phase': 0}, {'no_pulses': 650, 'f_0': 250000000.0, 'amplitude': 0.05001482699204792, 'f_sweep': 3000000.0, 'duration': 2.272295535603152e-05, 'delay': 1.4321789251686607e-05, 'phase': 0}], 'bounds_burn_back': [{'no_pulses': 160, 'f_0': 472302097.5196626, 'amplitude': 0.07396974413730217, 'freq_sweep': 187740.67323721855, 'duration': 1.550999487230298e-05, 'delay': 0.00017433386263766287, 'phase': 0}, {'no_pulses': 34, 'f_0': 10725081.256064394, 'amplitude': 0.032457348881016125, 'freq_sweep': 2948912.3704260103, 'duration': 3.668665459457938e-05, 'delay': 3.602842312344411e-05, 'phase': 0}, {'no_pulses': 12, 'f_0': 22308748

In [30]:
# Mutation Function

def mutate(population, GA_params, sequence_params_GA):
    mutation_rate = GA_params['mutation_rate']
    mutation_range = GA_params['mutation_range']
    
    for bound_key, individuals in population.items():
        for individual in individuals:
            for param, value in individual.items():
                if np.random.rand() < mutation_rate:
                    bound_param = sequence_params_GA[bound_key][param]
                    
                    if isinstance(bound_param, tuple) and len(bound_param) == 2:
                        lower_bound, upper_bound = bound_param
                        range_span = upper_bound - lower_bound
                        mutation_amount = range_span * mutation_range * (np.random.uniform(-1, 1))
                        
                        if isinstance(value, int):
                            new_value = int(value + mutation_amount)
                        else:
                            new_value = value + mutation_amount
                        
                        # Ensure the new value is within bounds
                        new_value = np.clip(new_value, lower_bound, upper_bound)
                        individual[param] = new_value
                    else:
                        # If the parameter is a constant, do not mutate
                        individual[param] = value
                        
    return population

# Example
mutated_population = mutate(sequence_params, GA_params, sequence_params_GA)

print("\nMutated Population :\n", mutated_population)


Mutated Population :
 {'bounds_pit': [{'no_pulses': np.int64(62), 'f_0': 250000000.0, 'amplitude': np.float64(0.08068585827645695), 'f_sweep': 3000000.0, 'duration': np.float64(3.9813315697478616e-05), 'delay': 7.09984570997758e-05, 'phase': 0}, {'no_pulses': np.int64(868), 'f_0': 250000000.0, 'amplitude': np.float64(0.029782602999267112), 'f_sweep': 3000000.0, 'duration': np.float64(4.839501091869794e-05), 'delay': np.float64(0.00022619258466468077), 'phase': 0}, {'no_pulses': np.int64(341), 'f_0': 250000000.0, 'amplitude': np.float64(0.06308751655715139), 'f_sweep': 3000000.0, 'duration': np.float64(2.437497845298358e-05), 'delay': 0.00028737326928689134, 'phase': 0}], 'bounds_burn_back': [{'no_pulses': np.int64(184), 'f_0': np.float64(248448838.50069225), 'amplitude': np.float64(0.15085982222943228), 'freq_sweep': np.float64(4160193.0727823833), 'duration': np.float64(8.652310229039752e-05), 'delay': np.float64(0.000254289446815816), 'phase': 0}, {'no_pulses': np.int64(472), 'f_0':

In [14]:
# Crossover breeding function

def crossover(parents, GA_params):
    crossover_rate = GA_params['crossover_rate']
    population_size = GA_params['population_size']
    num_parents = GA_params['num_parents']

    # Initialize offspring population dictionary
    offspring_population = {key: [] for key in parents.keys()}
    current_count = 0

    while current_count < population_size:
        # Randomly shuffle the order of parents
        shuffled_indices = np.random.permutation(num_parents)

        for i in range(0, num_parents, 2):
            if current_count >= population_size:
                break

            parent1_indices = shuffled_indices[i]
            if i + 1 < num_parents:
                parent2_indices = shuffled_indices[i + 1]
            else:
                parent2_indices = shuffled_indices[0]  # Wrap around if odd number of parents

            if np.random.rand() < crossover_rate:
                offspring1 = {}
                offspring2 = {}

                for key in parents.keys():
                    parent1 = parents[key][parent1_indices]
                    parent2 = parents[key][parent2_indices]

                    num_variables = len(parent1)
                    crossover_point = np.random.randint(1, num_variables)

                    offspring1[key] = {**{k: parent1[k] for k in list(parent1.keys())[:crossover_point]},
                                       **{k: parent2[k] for k in list(parent2.keys())[crossover_point:]}}
                    offspring2[key] = {**{k: parent2[k] for k in list(parent2.keys())[:crossover_point]},
                                       **{k: parent1[k] for k in list(parent1.keys())[crossover_point:]}}
                
                for key in offspring1.keys():
                    offspring_population[key].append(offspring1[key])
                current_count += 1
                if current_count < population_size:
                    for key in offspring2.keys():
                        offspring_population[key].append(offspring2[key])
                    current_count += 1
            else:
                # No crossover, offspring are copies of parents
                for key in parents.keys():
                    offspring_population[key].append(parents[key][parent1_indices])
                current_count += 1
                if current_count < population_size:
                    for key in parents.keys():
                        offspring_population[key].append(parents[key][parent2_indices])
                    current_count += 1

    # Ensure the offspring_population is trimmed to the population_size
    for key in offspring_population.keys():
        offspring_population[key] = offspring_population[key][:population_size]

    return offspring_population

# Example
offspring_population = crossover(selected_parents, GA_params)

print(offspring_population)

{'bounds_pit': [{'no_pulses': np.int64(134), 'f_0': 250000000.0, 'amplitude': np.float64(0.001), 'f_sweep': 3000000.0, 'duration': np.float64(8.882572651889464e-05), 'delay': np.float64(0.00013380047367810902), 'phase': 0}], 'bounds_burn_back': [{'no_pulses': np.int64(691), 'f_0': np.float64(51999253.751658455), 'amplitude': np.float64(0.056350462236971655), 'freq_sweep': np.float64(1755381.2962684345), 'duration': np.float64(5.1274859452921514e-05), 'delay': np.float64(0.00029737008930756366), 'phase': 0}]}


In [9]:
def parentselect(population, GA_params, fitness):
    num_parents = GA_params['num_parents']
    population_size = GA_params['population_size']
    
    # Calculate normalised fitness probabilities
    fit = fitness / np.sum(fitness)
    
    parents = {}
    
    for bound_key, individuals in population.items():
        individuals_array = np.array(individuals)
        population_indices = np.arange(len(individuals))
        
        # Weighted selection of parents
        selected_indices = np.random.choice(population_indices, size=num_parents, replace=False, p=fit)
        
        # Gather selected individuals
        parents[bound_key] = individuals_array[selected_indices].tolist()
    
    return parents

# Example
fitness = [0.1,0.1,0.9] # Nb. for population_size 3
selected_parents = parentselect(sequence_params, GA_params, fitness)
print(selected_parents)

{'bounds_pit': [{'no_pulses': 761, 'f_0': 250000000.0, 'amplitude': 0.20842370856902392, 'f_sweep': 3000000.0, 'duration': 2.7026530318061317e-05, 'delay': 0.0001386035361045073, 'phase': 0}], 'bounds_burn_back': [{'no_pulses': 996, 'f_0': 225036060.33874977, 'amplitude': 0.09867253215658939, 'freq_sweep': 1735161.4208328512, 'duration': 6.761045496478996e-05, 'delay': 0.00019741115827229643, 'phase': 0}], 'bounds_clean': [{'no_pulses': 74, 'f_0': 56661676.973667204, 'amplitude': 0.03924757281916513, 'freq_sweep': 2742035.568062731, 'duration': 3.564075177997523e-05, 'delay': 5.825331663126184e-05, 'phase': 0}]}


In [26]:
# Fitness function for calculating the echo NOTE this fitness function may not work look at the old function 
def fitness(echo_data_folder, input_data_folder, cross_data_folder):

    echo_data = pd.read_csv(echo_data_folder + '.csv')
    input_data = pd.read_csv(input_data_folder + '.csv')
    cross_data = pd.read_csv(cross_data_folder + '.csv')
    echo = echo_data['C_2 (V)']
    input = input_data['C_2 (V)']
    cross = cross_data['C_2 (V)']

    # For smoothing data
    kernal_size = 10
    kernal = np.transpose(np.ones(kernal_size)/kernal_size)

    cross = np.convolve(cross,kernal,mode='same')
    input = np.convolve(input,kernal,mode='same')
    echo = np.convolve(echo,kernal,mode='same')

    # Finding the area
    x = np.arange(len(echo))
    area_echo = np.trapezoid(echo,x)
    area_cross = np.trapezoid(cross,x)
    area_input = np.trapezoid(input,x)

    # Finding the efficiency
    fitness = (area_echo-area_input)/area_cross
    fitness = max(fitness, 0)

    return fitness

In [10]:
def optimal_solution(fit, params_GA_save, GA_params):
    index_of_max = np.unravel_index(np.argmax(fit, axis=None), fit.shape)
    i, j = index_of_max
    n = i * GA_params['population_size'] + j

    optimal_params = {key: params_GA_save[key][n] for key in params_GA_save}
    return optimal_params

In [11]:
# The implementation of GA code (This is pretty much what you will need to use)
import numpy as np
import pandas as pd

# Defining the GA parameters
GA_params = { 'population_size': 2,
             'num_parents': 1,
             'num_generations': 2,
             'crossover_rate': 0.8,
             'mutation_rate': 0.9,
             'mutation_range': 0.1}

sequence_params_GA = dict(
    bounds_pit = {'no_pulses': (1, 1001),
                 'f_0': 250E6,
                'amplitude': (0.001, 0.21),
                'f_sweep': 12E6/4,
                'duration': (10E-6, 100E-6),
                'delay': (10E-6, 300E-6),
                'phase': 0},

    bounds_burn_back = {'no_pulses': (1, 1001),
                'f_0': (10E5, 500E6),
                'amplitude': (0.001, 0.21),
                'freq_sweep': (0.1E-6, 5E6),
                'duration': (10E-6, 100E-6),
                'delay': (10E-6, 300E-6),
                'phase': 0},
)

sequence_params = initial_population(GA_params, sequence_params_GA)

params_GA_save = {key: [] for key in sequence_params}
fit = np.zeros((GA_params['num_generations']),(GA_params['population_size']))

for i in range(GA_params['num_generations']):

    for j in range(GA_params['population_size']):
        # compile and upload to HDAWG
        # Run HDAWG Sequence  
        #run code with this sequence_params 
        # = {key: sequence_params[key][j] for key in sequence_params}

        # save echo_data_folder, input_data_folder, cross_data_folder NOTE this function may need to be changed 
        fit[i,j] = fitness(echo_data_folder, input_data_folder, cross_data_folder)

    for key in sequence_params:
        params_GA_save[key].extend(sequence_params[key])

    parent_population = parentselect(sequence_params, GA_params, fit[i,:])
    new_sequence_params = crossover(parent_population, GA_params)
    sequence_params = mutate(new_sequence_params, GA_params, sequence_params_GA)

optimal_params = optimal_solution(fit, params_GA_save, GA_params)
print("\Optimal Params :\n", optimal_params)

NameError: name 'initial_population' is not defined